In [17]:
import openai
import pandas as pd
import re

# Your API key here
openai.api_key = "sk-aCxLYPM7ksEW5hRC0p80hQUs0LGuw5SQGSFgL1URxcT3BlbkFJG7J7YTe0SluXJZtU1ZMEr_y2VmtsZHIbn1nBsapmsA" 


In [18]:
reviews_general_selected  = pd.read_csv("C:/Users/Theresa/Downloads/filtered_reviews_general.csv")
reviews_additional_selected = pd.read_csv("C:/Users/Theresa/Downloads/filtered_reviews_additional.csv")

In [19]:
# merging the data
reviews = pd.merge(reviews_general_selected, reviews_additional_selected, on='review_id', how='outer')

# creating a subset
reviews_subset = reviews.head(10)

In [15]:
import pandas as pd
import re
import openai
from collections import defaultdict

def extract_topics(sentence):
    prompt = (
        "For the following sentence, identify all applicable categories: "
        "'food', 'service', 'atmosphere', 'price'. If no category applies, respond 'none'. "
        "Separate multiple categories with commas.\n\n"
        "Sentence: {sentence}\n\n"
        "Categories:"
    )

    try: 
        # Call the OpenAI API with the custom prompt
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(sentence=sentence)}
            ],
            max_tokens=150
        )
        
        generated_text = response.choices[0].message['content'].strip()
        return generated_text

    except Exception as e:
        print(f"Error in extracting topics: {e}")
        return None


def analyze_sentiment(sentences):
    sentiment_prompt = (
        "Rate the sentiment of the following sentences on a scale of 1 to 5, "
        "where 1 is 'okay' and 5 is 'very good'. Only provide the rating as a single number.\n\n"
        "Sentences: {sentences}\n\n"
        "Rating:"
    )

    try:
        # Combine all sentences for sentiment analysis
        combined_sentences = " ".join(sentences)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": sentiment_prompt.format(sentences=combined_sentences)}
            ],
            max_tokens=10
        )
        
        rating = response.choices[0].message['content'].strip()
        return int(rating)

    except Exception as e:
        print(f"Error in analyzing sentiment: {e}")
        return None


def process_reviews(reviews_subset):
    # Create a list to store results
    results = []

    for index, row in reviews_subset.iterrows():
        full_review = row['review_text']  # Full review text
        sentences = re.split(r'(?<=[.!?])\s+', full_review.strip())  # Split into sentences
        
        # Dictionary to store sentences by topic
        topic_groups = defaultdict(list)
        
        for sentence in sentences:
            topics = extract_topics(sentence)  # Extract topics from the sentence
            if topics and topics.lower() != 'none':
                for topic in topics.split(","):
                    topic = topic.strip().lower()
                    topic_groups[topic].append(sentence)
        
        # Initialize dictionary for review data
        review_data = {
            "full_review": full_review,
            "atmosphere_sentence": None,
            "atmosphere_rating": None,
            "service_sentence": None,
            "service_rating": None,
            "food_sentence": None,
            "food_rating": None,
            "price_sentence": None,
            "price_rating": None,
        }

        # Process each topic and its grouped sentences
        for topic, sentences_for_topic in topic_groups.items():
            sentiment_rating = analyze_sentiment(sentences_for_topic)  # Perform sentiment analysis
            if sentiment_rating:
                # Assign the first sentence for the topic and the sentiment rating
                review_data[f"{topic}_sentence"] = " ".join(sentences_for_topic)
                review_data[f"{topic}_rating"] = sentiment_rating
        
        # Append review data to results
        results.append(review_data)
    
    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Process the reviews and create the detailed DataFrame
detailed_reviews_df = process_reviews(reviews_subset)

# Display the resulting DataFrame
print(detailed_reviews_df)


Error in analyzing sentiment: invalid literal for int() with base 10: 'Rating: 5'
Error in analyzing sentiment: invalid literal for int() with base 10: 'Rating: 4'
Error in analyzing sentiment: invalid literal for int() with base 10: 'Rating: 4'
                                         full_review  \
0  Im Veggi in Tübingen haben wir einen gemütlich...   
1  War dort heute zum ersten Mal essen, da meine ...   
2  Übersichtliche Speisekarte und somit einfach k...   
3  Veggie in Tübingen 100 % Empfehlung. Hier ist ...   
4  Der Laden wurde uns empfohlen und wir wurden n...   
5                         Sehr gemütlich und lecker.   
6  Veggie leider nur noch ein trüber Schatten des...   
7  Wollte dieses Restaurant schon lange mal auspr...   
8  Leckere Falafel mit super guten Dips! Schöne k...   
9  Die Suppe war leider viel zu dick für unseren ...   

                                 atmosphere_sentence  atmosphere_rating  \
0  Im Veggi in Tübingen haben wir einen gemütlich...          